<a href="https://colab.research.google.com/github/Purva-Chaudhari/mdanalysis/blob/master/base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Purva-Chaudhari/mdanalysis.git

Cloning into 'mdanalysis'...
remote: Enumerating objects: 312, done.
remote: Counting objects: 100% (312/312), done.
remote: Compressing objects: 100% (215/215), done.
remote: Total 78241 (delta 256), reused 137 (delta 92), pack-reused 77929
Receiving objects: 100% (78241/78241), 234.06 MiB | 31.57 MiB/s, done.
Resolving deltas: 100% (63746/63746), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd mdanalysis/

/content/mdanalysis


In [4]:
!pip install --upgrade MDAnalysis


     |████████████████████████████████| 3.7MB 2.8MB/s 
     |████████████████████████████████| 2.3MB 30.3MB/s 
     |████████████████████████████████| 2.0MB 55.6MB/s 
     |████████████████████████████████| 317kB 58.3MB/s 
  Created wheel for MDAnalysis: filename=MDAnalysis-0.20.1-cp36-cp36m-linux_x86_64.whl size=4577674 sha256=dc2616ef59eb6e137559f42267d4fed8d3222d7a0b5d698ce04deb2096a312f4
  Stored in directory: /root/.cache/pip/wheels/30/8e/3b/fae2c19789bb9cd83fe34942dcbc1bafb26ea6433f1ab464ac
Successfully built MDAnalysis


In [5]:
!pip install --upgrade MDAnalysisTests

     |████████████████████████████████| 47.6MB 92kB/s 
     |████████████████████████████████| 296kB 58.5MB/s 
  Created wheel for MDAnalysisTests: filename=MDAnalysisTests-0.20.1-py2.py3-none-any.whl size=47888370 sha256=5be86531a64fcd6e842b7bddeab787d812d84caf3d513b7bb3a7e14a1e4a14a9
  Stored in directory: /root/.cache/pip/wheels/e6/91/bf/31b0f76a26bfcf5a3569b0f9822f3f0e2094790ff39d5b1f74
Successfully built MDAnalysisTests


In [6]:
import MDAnalysis as mda
from MDAnalysis.tests.datafiles import PSF, DCD, GRO, XTC

print(mda.Universe(PSF, DCD))
print(mda.__version__)

<Universe with 3341 atoms>
0.20.1


In [7]:
u2 = mda.Universe(GRO, XTC, XTC, all_coordinates=False, continuous=False)
print([int(ts.time) for ts in u2.trajectory])

[0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 0, 100, 200, 300, 400, 500, 600, 700, 800, 900]


In [8]:
import MDAnalysis as mda
from MDAnalysis.tests.datafiles import PDB, GRO, XTC
u1 = mda.Universe(GRO, XTC, XTC, all_coordinates=True)
u1.trajectory

/usr/local/lib/python3.6/dist-packages/MDAnalysis/coordinates/base.py:837: UserWarning: Reader has no dt information, set to 1.0 ps
  warnings.warn("Reader has no dt information, set to 1.0 ps")


<ChainReader containing adk_oplsaa.gro, adk_oplsaa.xtc, adk_oplsaa.xtc with 21 frames of 47681 atoms>

In [11]:
from __future__ import absolute_import
import warnings
import six
from six.moves import range, zip
import inspect
import logging
import itertools
import numpy as np
from MDAnalysis import coordinates
from MDAnalysis.core.groups import AtomGroup
from MDAnalysis.lib.log import ProgressMeter

logger = logging.getLogger(__name__)

class AnalysisBase(object):
    def __init__(self, trajectory, verbose=False, **kwargs):
        self._trajectory = trajectory
        self._verbose = verbose

    def _setup_frames(self, trajectory, start=None, stop=None, step=None):
        self._trajectory = trajectory
        start, stop, step = trajectory.check_slice_indices(start, stop, step)
        if start is not None:
          self._start = start
        elif stop is not None:
          self._stop = stop
        elif step is not None:
          self._step = step
        else:
          warnings.warn("Deprecation occured", DeprecationWarning)
        self.n_frames = len(range(start, stop, step))
        #print("n_frames = ",self.n_frames)
        interval = int(self.n_frames // 100)
        if interval == 0:
            interval = 1

        verbose = getattr(self, '_verbose', False)
        print("verbose1 = ", verbose)
        self._pm = ProgressMeter(self.n_frames if self.n_frames else 1,
                                 interval=interval, verbose=verbose)
        print("verbose2 = ", verbose)

    def _single_frame(self):
        raise NotImplementedError("Only implemented in child classes")

    def _prepare(self):
        pass # pylint: disable=unnecessary-pass

    def _conclude(self):
        pass # pylint: disable=unnecessary-pass

    def run(self, start=None, stop=None, step=None, verbose=None):     
        logger.info("Choosing frames to analyze")
        # if verbose unchanged, use class default
        verbose = getattr(self, '_verbose', False) if verbose is None else verbose
        print(verbose)
        print("self verbose "self._verbose)
        self._setup_frames(self._trajectory, start, stop, step)
        logger.info("Starting preparation")
        self._prepare()
        for i, ts in enumerate(
                self._trajectory[self.start:self.stop:self.step]):
            self._frame_index = i
            self._ts = ts
            # logger.info("--> Doing frame {} of {}".format(i+1, self.n_frames))
            self._single_frame()
            self._pm.echo(self._frame_index)
        logger.info("Finishing up")
        self._conclude()
        return self

class AnalysisFromFunction(AnalysisBase):    
    def __init__(self, function, trajectory=None, *args, **kwargs):     
        if (trajectory is not None) and (not isinstance(
                trajectory, coordinates.base.ProtoReader)):
            args = (trajectory,) + args
            trajectory = None

        if trajectory is None:
            # all possible places to find trajectory
            for arg in itertools.chain(args, six.itervalues(kwargs)):
                if isinstance(arg, AtomGroup):
                    trajectory = arg.universe.trajectory
                    break

        if trajectory is None:
            raise ValueError("Couldn't find a trajectory")

        self.function = function
        self.args = args

        self.kwargs = kwargs

        super(AnalysisFromFunction, self).__init__(trajectory)

    def _prepare(self):
        self.results = []

    def _single_frame(self):
        self.results.append(self.function(*self.args, **self.kwargs))

    def _conclude(self):
        self.results = np.asarray(self.results)


def analysis_class(function):   
    class WrapperClass(AnalysisFromFunction):
        def __init__(self, trajectory=None, *args, **kwargs):
            super(WrapperClass, self).__init__(function, trajectory,
                                               *args, **kwargs)

    return WrapperClass

def _filter_baseanalysis_kwargs(function, kwargs):  
    try:
        # pylint: disable=deprecated-method
        base_argspec = inspect.getfullargspec(AnalysisBase.__init__)
    except AttributeError:
        # pylint: disable=deprecated-method
        base_argspec = inspect.getargspec(AnalysisBase.__init__)

    n_base_defaults = len(base_argspec.defaults)
    base_kwargs = {name: val
                   for name, val in zip(base_argspec.args[-n_base_defaults:],
                                        base_argspec.defaults)}

    try:
        # pylint: disable=deprecated-method
        argspec = inspect.getfullargspec(function)
    except AttributeError:
        # pylint: disable=deprecated-method
        argspec = inspect.getargspec(function)

    for base_kw in six.iterkeys(base_kwargs):
        if base_kw in argspec.args:
            raise ValueError(
                "argument name '{}' clashes with AnalysisBase argument."
                "Now allowed are: {}".format(base_kw, base_kwargs.keys()))

    base_args = {}
    for argname, default in six.iteritems(base_kwargs):
        base_args[argname] = kwargs.pop(argname, default)

    return base_args, kwargs

SyntaxError: ignored

In [12]:
obj = AnalysisBase(u2.trajectory)
obj._setup_frames(u2.trajectory,None,None,None)
#obj._setup_frames(u2.trajectory,0,10,2)
print(obj.start)
print(obj.stop)
print(obj.step)
obj.run(verbose = True)

verbose1 =  False
verbose2 =  False


AttributeError: ignored

In [0]:
!git diff

In [0]:
#issue 2206
from MDAnalysis.analysis.lineardensity import LinearDensity
from MDAnalysis.tests.datafiles import PSF_TRICLINIC, DCD_TRICLINIC

u = mda.Universe(PSF_TRICLINIC, DCD_TRICLINIC)
ldens = LinearDensity(u.atoms, verbose=True)

ldens.run(start=0, stop=10)
print(ldens.start, ldens.stop)

ldens.run(start=0, stop=5, verbose=False)
print(ldens.start, ldens.stop)

Step    10/10 [100.0%]
Step    10/10 [100.0%]


0 10
0 10


In [13]:
!git pull

Already up to date.


In [15]:
!git fetch origin
!git checkout -b yourbranch origin/yourbranch
!git merge master

fatal: 'origin/yourbranch' is not a commit and a branch 'yourbranch' cannot be created from it
merge: master - not something we can merge

Did you mean this?
	tdata-0-7-4
